In [1]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching ip_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s ip_Scene"

Manim Community v0.18.0.post0

Jupyter Capture Output v0.0.11


In [157]:
class InclinedPlane(Mobject):
    def __init__(self, starting_center, height, width, **kwargs):
        super().__init__(**kwargs)

        self.ax_center = starting_center            # this is good for exaxtly nothing because manim doesn't put ax origin into center [desgracia]
        self.wedge_height = height
        self.wedge_width = width
        self.angle = np.arctan(height / width)


        # coordinate system c
        self.ax_c = Axes(x_range = (-1, self.wedge_width, 1), y_range = (-self.wedge_height, 1, 1), x_length = self.wedge_width+1, y_length = self.wedge_height+1,
            axis_config = {'color': BLACK, "tip_width": 0.15, "tip_height": 0.15}
        ).move_to(self.ax_center)
        self.ax_c_xlabel = self.ax_c.get_x_axis_label(Tex(r"$x$", font_size = 36, color = BLACK)).shift(0.1 *LEFT)
        self.ax_c_ylabel = self.ax_c.get_y_axis_label(Tex(r"$y$", font_size = 36, color = BLACK))
        self.add(self.ax_c, self.ax_c_xlabel, self.ax_c_ylabel)

        inclined_plane = Line(start = self.ax_c.c2p(0, 0, 0), end = self.ax_c.c2p(self.wedge_width, -self.wedge_height, 0), color = BLACK, stroke_opacity = 0.5)
        inclined_plane_fill = Polygon(self.ax_c.c2p(0, 0, 0), self.ax_c.c2p(self.wedge_width, -self.wedge_height, 0), self.ax_c.c2p(0, -self.wedge_height, 0), fill_color = BLACK, fill_opacity = 0.125, stroke_opacity = 0)
        self.add(inclined_plane, inclined_plane_fill)


    # coordinate system c'
    def get_c_star(self):
        self.ax_c_star = Axes(x_range = (-1, self.wedge_width+1, 1), y_range = (-self.wedge_height-1, 1, 1), x_length = self.wedge_width+2, y_length = self.wedge_height+2,
            axis_config = {'color': BLACK, "tip_width": 0.15, "tip_height": 0.15}
        ).move_to(self.ax_center).rotate(about_point = self.ax_c.c2p(0, 0, 0), angle = -self.angle)
        self.ax_c_star_xlabel = self.ax_c_star.get_x_axis_label(Tex(r"$x'$", font_size = 36, color = BLACK)).shift(0.1 *LEFT)
        self.ax_c_star_ylabel = self.ax_c_star.get_y_axis_label(Tex(r"$y'$", font_size = 36, color = BLACK))
        # self.add(self.ax_c_star)#, self.ax_c_star_xlabel, self.ax_c_star_ylabel)     

        return VGroup(self.ax_c_star)


    # waggon rolling down the plane
    def get_waggon(self, t):
        waggon_pos = self.ax_c.c2p(5*t**2 * np.sin(self.angle) * np.cos(self.angle), -5*t**2 * np.sin(self.angle) * np.sin(self.angle), 0) + 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        waggon = Rectangle(height = 0.25, width = 0.5, color = BLACK, fill_color = BLACK, fill_opacity = 0.5
            ).move_to(waggon_pos).rotate(about_point = waggon_pos, angle = -self.angle)
        # gravitation vector downwards
        waggon.gravitation = Line(start = waggon_pos, end = waggon_pos + 1.5*DOWN, color = BLACK).add_tip(tip_length = 0.15, tip_width = 0.15)
        waggon.perp = Line(start = waggon_pos, end = waggon_pos + np.cos(self.angle)*(np.cos(self.angle)*1.5*DOWN - np.sin(self.angle)*1.5*RIGHT), color = BLACK).add_tip(tip_length = 0.15, tip_width = 0.15)
        waggon.line = Line(start = waggon_pos, end = waggon_pos + np.sin(self.angle)*(np.sin(self.angle)*1.5*DOWN + np.cos(self.angle)*1.5*RIGHT), color = BLACK).add_tip(tip_length = 0.15, tip_width = 0.15)
        return waggon
    

    # ball getting ejected
    def get_ball(self, t):
        v0 = 6                                      # starting velocity
        ejection_time = 0.75
        # coordinate of the waggon
        waggon_c = np.array([5*t**2 * np.sin(self.angle) * np.cos(self.angle), -5*t**2 * np.sin(self.angle) * np.sin(self.angle), 0])
        # additional ejection position change
        ball_increment_c = np.array([v0 * (t-ejection_time) * np.sin(self.angle), -5*(t-ejection_time)**2 + v0 * (t-ejection_time) * np.cos(self.angle), 0])
        # speed and position of the waggon at ejection
        waggon_eject_pos = np.array([5*ejection_time**2 * np.sin(self.angle) * np.cos(self.angle), -5*ejection_time**2 * np.sin(self.angle) * np.sin(self.angle), 0])
        waggon_eject_speed = np.array([10*ejection_time * np.sin(self.angle) * np.cos(self.angle), -10*ejection_time * np.sin(self.angle) * np.sin(self.angle), 0])
        # different time sections
        if (t < ejection_time):
            ball_pos = self.ax_c.c2p(*waggon_c) + 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        elif (t > ejection_time + 2*v0 / (10*np.cos(self.angle))):
            ball_pos = self.ax_c.c2p(*waggon_c) + 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        else:
            ball_pos = self.ax_c.c2p(*(waggon_eject_pos + waggon_eject_speed*(t-ejection_time) + ball_increment_c)) + 0.25/2*(np.cos(self.angle)*UP + np.sin(self.angle)*RIGHT)
        ball = Circle(radius = 0.125, color = PURE_RED, fill_color = PURE_RED, fill_opacity = 0.5).move_to(ball_pos)
        return ball



In [158]:
%%capture_video --path "animations/inclined_plane/inclined_plane.mp4"
%%manim -qm --fps 60 $video_scene


class ip_Scene(Scene):
    def construct(self):
        self.camera.background_color = WHITE

        center = np.array([0, 0, 0])

        inclined_plane = InclinedPlane(center, height = 4, width = 10)
        self.add(inclined_plane)

        waggon = inclined_plane.get_waggon(0)
        ball = inclined_plane.get_ball(0)
        waggon.getter = inclined_plane.get_waggon
        ball.getter = inclined_plane.get_ball
        self.add(waggon, ball)


        def physics_updater(object):
            t = time.get_value()
            get_object = object.getter
            object.become(get_object(t))


        time = ValueTracker(0)
        waggon.add_updater(physics_updater)
        ball.add_updater(physics_updater)
        self.play(time.animate.set_value(1.5), rate_func= linear, run_time = 3)
        # vectors
        waggon = inclined_plane.get_waggon(time.get_value())
        ball = inclined_plane.get_ball(time.get_value())
        waggon_gravitation_vector = waggon.gravitation
        waggon_perp_vector = waggon.perp
        waggon_line_vector = waggon.line
        self.add(waggon_gravitation_vector, waggon_perp_vector, waggon_line_vector)
        self.wait(3)

Output saved by overwring previous file at animations/inclined_plane/inclined_plane.mp4.
